# Имитационное и статистическое моделирование
## Лабораторная № 2
## Павел Акопян, 2 группа

Содержание:
1. Реализация теста MonoBit, Linear Complexity, Matrix Rank, Serial из группы тестов NIST для проверки качества генерируемой псевдослучайной последовательности (из 0 и 1).
2. Реализация регистра сдвига с линейной обратной связью и самосжиющегося генератора.
3. Проверка качества последовательностей.
4. Реализация алгоритма генерации и шифрования A5/1.
5. Применение A5/1 для шифрования текстовых файлов.

Загрузка необходимых пакетов

In [6]:
import scipy.special as spc
import scipy.stats as sst
import pandas as pd
import numpy
import math
import copy
import os

Загрузка библиотеки, позволяющей удобно работать с битовыми массивами

In [7]:
from bitstring import BitArray, Bits

Настройка формата вывода

In [8]:
pd.set_option( 'display.precision', 3)
pd.set_option('chop_threshold', .00001)
numpy.set_printoptions(precision = 4, suppress=True)
pd.set_eng_float_format(accuracy=3,use_eng_prefix=False)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Monobit

In [9]:
def monobit(bin_data):
    count = 0
    for char in bin_data:
        if char == 0:
            count -= 1
        else:
            count += 1
    sobs = count / math.sqrt(len(bin_data))
    p_val = spc.erfc(math.fabs(sobs) / math.sqrt(2))
    return p_val

# Rank Test

In [5]:
class BinaryMatrix:
    def __init__(self, matrix, rows, cols):
        self.M = rows
        self.Q = cols
        self.A = matrix
        self.m = min(rows, cols)

    def compute_rank(self, verbose=False):
        if verbose:
            print("Original Matrix\n", self.A)

        i = 0
        while i < self.m - 1:
            if self.A[i][i] == 1:
                self.perform_row_operations(i, True)
            else:
                found = self.find_unit_element_swap(i, True)
                if found == 1:
                    self.perform_row_operations(i, True)
            i += 1

        if verbose:
            print("Intermediate Matrix\n", self.A)

        i = self.m - 1
        while i > 0:
            if self.A[i][i] == 1:
                self.perform_row_operations(i, False)
            else:
                if self.find_unit_element_swap(i, False) == 1:
                    self.perform_row_operations(i, False)
            i -= 1

        if verbose:
            print("Final Matrix\n", self.A)

        return self.determine_rank()

    def perform_row_operations(self, i, forward_elimination):
        if forward_elimination:
            j = i + 1
            while j < self.M:
                if self.A[j][i] == 1:
                    self.A[j, :] = (self.A[j, :] + self.A[i, :]) % 2
                j += 1
        else:
            j = i - 1
            while j >= 0:
                if self.A[j][i] == 1:
                    self.A[j, :] = (self.A[j, :] + self.A[i, :]) % 2
                j -= 1

    def find_unit_element_swap(self, i, forward_elimination):
        row_op = 0
        if forward_elimination:
            index = i + 1
            while index < self.M and self.A[index][i] == 0:
                index += 1
            if index < self.M:
                row_op = self.swap_rows(i, index)
        else:
            index = i - 1
            while index >= 0 and self.A[index][i] == 0:
                index -= 1
            if index >= 0:
                row_op = self.swap_rows(i, index)
        return row_op

    def swap_rows(self, i, ix):
        temp = copy.copy(self.A[i, :])
        self.A[i, :] = self.A[ix, :]
        self.A[ix, :] = temp
        return 1

    def determine_rank(self):
        rank = self.m
        i = 0
        while i < self.M:
            all_zeros = 1
            for j in range(self.Q):
                if self.A[i][j] == 1:
                    all_zeros = 0
            if all_zeros == 1:
                rank -= 1
            i += 1
        return rank

In [36]:
def matrix_rank( bin_data, q=32):
    shape = (q, q)
    n = len(bin_data)
    block_size = int(q * q)
    num_m = int(math.floor(n / (q * q)))
    block_start, block_end = 0, block_size
    # print(q, n, num_m, block_size)

    if num_m > 0:
        max_ranks = [0, 0, 0]
        for im in range(num_m):
            block_data = bin_data[block_start:block_end]
            block = numpy.zeros(len(block_data))
            for i in range(len(block_data)):
                if block_data[i] == 1:
                    block[i] = 1.0
            m = block.reshape(shape)
            ranker = BinaryMatrix(m, q, q)
            rank = ranker.compute_rank()
            # print(rank)
            if rank == q:
                max_ranks[0] += 1
            elif rank == (q - 1):
                max_ranks[1] += 1
            else:
                max_ranks[2] += 1
            # Update index trackers
            block_start += block_size
            block_end += block_size

        piks = [1.0, 0.0, 0.0]
        for x in range(1, 50):
            piks[0] *= 1 - (1.0 / (2 ** x))
        piks[1] = 2 * piks[0]
        piks[2] = 1 - piks[0] - piks[1]

        chi = 0.0
        for i in range(len(piks)):
            chi += pow((max_ranks[i] - piks[i] * num_m), 2.0) / (piks[i] * num_m)
        p_val = math.exp(-chi / 2)
        return p_val
    else:
        return -1.0

In [37]:
matrix_rank(ssh)

1024


0.23335899811528313

In [38]:
matrix_rank(e[0:100000])

1024


0.5320686217466569

# Linear Complexity Test

In [85]:
def linear_complexity(bin_data, block_size=500):
    dof = 6
    piks = [0.01047, 0.03125, 0.125, 0.5, 0.25, 0.0625, 0.020833]

    t2 = (block_size / 3.0 + 2.0 / 9) / 2 ** block_size
    mean = 0.5 * block_size + (1.0 / 36) * (9 + (-1) ** (block_size + 1)) - t2

    num_blocks = int(len(bin_data) / block_size)
    print num_blocks
    if num_blocks > 1:
        block_end = block_size
        block_start = 0
        blocks = []
        for i in range(num_blocks):
            blocks.append(bin_data[block_start:block_end])
            block_start += block_size
            block_end += block_size

        complexities = []
        for block in blocks:
            complexities.append(berlekamp_massey_algorithm(block))

        t = ([(((-1) ** block_size) * (chunk - mean) + 2.0 / 9) for chunk in complexities])
        #t = ([-1.0 * (((-1) ** block_size) * (chunk - mean) + 2.0 / 9) for chunk in complexities])
        #vg = numpy.histogram(t, bins=[-9999999999, -2.50000000001,
        #                              -1.50000000001, -0.50000000001, 
        #                              0.50000000001, 1.50000000001, 
        #                              2.50000000001, 99999999999])[0][::-1]
        vg = numpy.histogram(t, bins=[-9999999999, -2.50000000001,
                                      -1.50000000001, -0.50000000001, 
                                      0.50000000001, 1.50000000001, 
                                      2.50000000001, 99999999999])[0]
        
        im = ([((float(vg[ii]) - float(num_blocks * piks[ii])) ** 2) / (num_blocks * piks[ii]) for ii in range(7)])
        print vg
        chi_squared = 0.0
        for i in range(len(piks)):
            chi_squared += im[i]
        print chi_squared
        p_val = spc.gammaincc(dof / 2.0, chi_squared / 2.0)
        return p_val
    else:
        return -1.0

def berlekamp_massey_algorithm( block_data):
    n = len(block_data)
    c = numpy.zeros(n)
    b = numpy.zeros(n)
    c[0], b[0] = 1, 1
    l, m, i = 0, -1, 0
    int_data = [int(el) for el in block_data]
    while i < n:
        v = int_data[(i - l):i]
        v = v[::-1]
        cc = c[1:l + 1]
        d = (int_data[i] + numpy.dot(v, cc)) % 2
        if d == 1:
            temp = copy.copy(c)
            p = numpy.zeros(n)
            for j in range(0, l):
                if b[j] == 1:
                    p[j + i - m] = 1
            c = (c + p) % 2
            if l <= 0.5 * i:
                l = i + 1 - l
                m = i
                b = temp
        i += 1
    return l

In [86]:
linear_complexity(e[0:1000000])

2000
[  21   52  250 1006  492  135   44]
2.85891535567


0.82633477040383041

In [87]:
linear_complexity(e[0:999999])

1999
[  21   52  250 1005  492  135   44]
2.84873374904


0.82757885627625838

# Serial Test

In [13]:
def serial(bin_data, pattern_length=16, method="first"):
    n = len(bin_data)
    # Add first m-1 bits to the end
    bin_data += bin_data[:pattern_length-1]

    # Get max length one patterns for m, m-1, m-2
    max_pattern = ''
    for i in range(pattern_length):
        max_pattern += '1'

    # Keep track of each pattern's frequency (how often it appears)
    vobs_one = numpy.zeros(int(max_pattern[0:pattern_length], 2) + 1)
    vobs_two = numpy.zeros(int(max_pattern[0:pattern_length-1], 2) + 1)
    vobs_thr = numpy.zeros(int(max_pattern[0:pattern_length-2], 2) + 1)

    for i in range(n):
        # Work out what pattern is observed
        #print bin_data[i:i + pattern_length].bin
        vobs_one[int(bin_data[i:i + pattern_length].bin, 2)] += 1
        vobs_two[int(bin_data[i:i + pattern_length-1].bin, 2)] += 1
        vobs_thr[int(bin_data[i:i + pattern_length-2].bin, 2)] += 1

    vobs = [vobs_one, vobs_two, vobs_thr]
    #print vobs
    sums = numpy.zeros(3)
    for i in range(3):
        for j in range(len(vobs[i])):
            sums[i] += pow(vobs[i][j], 2)
        sums[i] = (sums[i] * pow(2, pattern_length-i)/(n)) - (n)

    # Calculate the test statistics and p values
    del1 = sums[0] - sums[1]
    del2 = sums[0] - 2.0 * sums[1] + sums[2]
    #print del1/2.0, del2/2.0
    #print pow(2, pattern_length-1)/2.0,pow(2, pattern_length-2)/2.0
    p_val_one = spc.gammaincc(pow(2.0, pattern_length-1)/2.0,del1/2.0)
    p_val_two = spc.gammaincc(pow(2.0, pattern_length-2)/2.0, del2/2.0)
    #print p_val_one, p_val_two
    # For checking the outputs
    if method == "first":
        return p_val_one
    else:
        # I am not sure if this is correct, but it makes sense to me.
        return min(p_val_one, p_val_two)

In [14]:
serial(e[0:1000000],16)

0.76389203335709599

In [90]:
serial(BitArray('0b0011011101'),3)

001
011
110
101
011
111
110
101
010
100
[array([ 0.,  1.,  1.,  2.,  1.,  2.,  2.,  1.]), array([ 1.,  3.,  3.,  3.]), array([ 4.,  6.])]
0.8 0.4
2.0 1.0
0.808792135411 0.670320046036


0.80879213541099859

In [15]:
print spc.gammaincc(2.0,0.8)
print spc.gammaincc(1.0,0.4)

0.808792135411
0.670320046036


In [ ]:
spc.

# LFSR

In [20]:
LFSR(50).bin

'01111111101110000100000111100100000111010000111111'

In [16]:
def LFSR(n):
    reg = 0x4F043BFD
    s = BitArray()
    for i in xrange(n):
        reg = ((((reg>>31)^(reg>>30)^(reg>>29)^(reg>>27)^(reg>>25)^(reg)) & 1) <<31) | (reg>>1)
        s.append('0b'+str(reg& 1)) 
    reg = 1
    return s

In [10]:
def SSH(n):
    reg = 0xFFFFFFFF
    s = BitArray()
    i = 0
    while True:
        reg = ((((reg>>31)^(reg>>30)^(reg>>29)^(reg>>27)^(reg>>25)^(reg)) & 1) <<31) | (reg>>1)
        a = reg& 1
        reg = ((((reg>>31)^(reg>>30)^(reg>>29)^(reg>>27)^(reg>>25)^(reg)) & 1) <<31) | (reg>>1)
        b = reg& 1
        if a:
            if b:
                s.append('0b1') 
            else:
                s.append('0b0')    
            i = i+1
        if i==n:
            break;
        
    reg = 0xFFFFFFFF
    return s

# Генератор A5

In [11]:
def LFSR_m(n,m):
    reg = BitArray('0b'+'0'.zfill(m))
    reg.invert()
    one = BitArray('0b'+'1'.zfill(m))
    s = BitArray()
    for i in xrange(32):
        reg = ((((reg>>31)^(reg>>30)^(reg>>29)^(reg>>27)^(reg>>25)^(reg)) & one) <<31) | (reg>>1)
    for i in xrange(n):
        reg = ((((reg>>31)^(reg>>30)^(reg>>29)^(reg>>27)^(reg>>25)^(reg)) & one) <<31) | (reg>>1)

        s.append('0b'+str(int((reg& one)[-1])) )  
    return s

In [12]:
class A5generator:
    def __init__(self, verbose = True):
        self.verbose = verbose
        self.reg1= BitArray('0b'+'1'*19) # 19 bit register
        self.reg2= BitArray('0b'+'1'*22) # 22 bit register
        self.reg3= BitArray('0b'+'1'*23) # 23 bit register
    def __onesRegisters(self):
        self.reg1= BitArray('0b'+'1'*19) # 19 bit register
        self.reg2= BitArray('0b'+'1'*22) # 22 bit register
        self.reg3= BitArray('0b'+'1'*23) # 23 bit register        
    def __clockLFSR1(self, inputBit =0):
        reg1XOR = int(((self.reg1>>5)^(self.reg1>>2)^(self.reg1>>1)^(self.reg1))[-1])^inputBit
        self.reg1 = self.reg1>>1
        self.reg1[0]= reg1XOR
    def __clockLFSR2(self, inputBit =0):
        reg2XOR = int(((self.reg2>>1)^(self.reg2))[-1])^inputBit
        self.reg2 = self.reg2>>1
        self.reg2[0]= reg2XOR
    def __clockLFSR3(self, inputBit =0):
        reg3XOR = int(((self.reg3>>15)^(self.reg3>>2)^(self.reg3>>1)^(self.reg3))[-1])^inputBit
        self.reg3 = self.reg3>>1
        self.reg3[0]= reg3XOR
    def __majorityFunc(self):
        x = self.reg1[8]
        y = self.reg2[10]
        z = self.reg3[10]
        return (x&y)|(x&z)|(y&z)
    def __emptyClocks(self, numIt = 100): #initial clocks without output
        for i in range(0,numIt):
            major = self.__majorityFunc()
            if self.reg1[8]==major:
                self.__clockLFSR1()
            if self.reg2[10]==major:
                self.__clockLFSR2()
            if self.reg3[10]==major:
                self.__clockLFSR3()
    
    def __mClocks(self, numIt): # clocks for random stream
        output = BitArray()
        for i in range(0,numIt):
            rez = self.reg1[-1]^self.reg2[-1]^self.reg3[-1]
            output.append('0b'+str(int(rez)))
            major = self.__majorityFunc()
            if self.reg1[8]==major:
                self.__clockLFSR1()
            if self.reg2[10]==major:
                self.__clockLFSR2()
            if self.reg3[10]==major:
                self.__clockLFSR3()
        return output
    def A5generating(self,n=100):
        self.__onesRegisters()
        self.__emptyClocks()
        output = self.__mClocks(n)
        if self.verbose:print output
        return output     
        print "================================================================"
        print "                                                                "
        print "================================================================"

# Проверка всех тестов на всех последовательностях

Загрузка из текстовых файлов ранее сгенерированных бинарных последовательностей, а также последовательности e

In [10]:
def loadBinFromFile(filename):
    f = open(filename, 'rb')
    a =f.read()[:-1]
    f.close()
    return BitArray('0b'+a)

In [11]:
e = loadBinFromFile(filename = 'e.txt')
t1 = loadBinFromFile(filename = 'txt1b.txt')
t2 = loadBinFromFile(filename = 'txt2b.txt')
t3 = loadBinFromFile(filename = 'txt3b.txt')
t4 = loadBinFromFile(filename = 'txt4b.txt')

Генерируем последовательности при помощи новых генераторов

In [15]:
g= A5generator(verbose=False)
a5 = g.A5generating(len(t4))
lfsr = LFSR(len(t4))
ssh = SSH(len(t4)) 

Объединяем все последовательности в один объект

In [16]:
allSequences = [e,t1,t2,t3,t4,lfsr,ssh,a5]

Все тесты в объединяются одну функцию, выдающую таблицу результатов

In [17]:
def applyAllTests(allSequences):
    mb = map(lambda x: monobit(x),allSequences)
    lc = map(lambda x: linear_complexity(x),allSequences)
    mr = map(lambda x: matrix_rank(x),allSequences)
    s = map(lambda x: serial(x),allSequences)
    head0 = ["e","t1","t2","t3","t4","32-lfsr","self-shrink","a5"]
    return pd.DataFrame({"monobit":mb,"linear complexity":lc,"matrix rank":mr,
                         "serial": s, "data name":head0
                        })

In [18]:
tests_results= applyAllTests(allSequences)

In [19]:
tests_results.set_index('data name')

,linear complexity,matrix rank,monobit,serial
data name,,,,
e,0.828,0.306,0.953,0.764
t1,0.000,0.000,0.044,0.000
t2,0.475,0.475,0.944,0.000
t3,0.515,0.544,0.046,0.000
t4,0.815,0.052,0.992,0.000
32-lfsr,0.000,0.000,0.928,0.410
self-shrink,0.908,0.233,0.070,0.992
a5,0.941,0.873,0.841,0.711


# Шифрование A5/1 (как в стандарте GSM)

In [20]:
class A5cipher:
    def __init__(self, sessionKey= '0b1010101110111000111111001010101010111101110101110111001100101110', verbose = True):
        self.verbose = verbose
        self.key = BitArray(sessionKey)
        self.reg1= BitArray('0b'+'0'*19) # 19 bit register
        self.reg2= BitArray('0b'+'0'*22) # 22 bit register
        self.reg3= BitArray('0b'+'0'*23) # 23 bit register
        self.frameCounterSize = 22 # Bits per frame counter 
    def loadText(self, text = "A like kittens!"):
        self.initText = text
        self.encodedBinStream = BitArray()
        self.binStream = self.__text2bin(text)
        self.Decoded = self.__bin2text(self.binStream.bin)
        if self.verbose: print "Загруженный текст:\n"+self.Decoded
        if self.verbose: print "Бинарное представление загруженного текста:\n"+str(self.binStream.bin)
        self.__makePackets()
        if self.verbose: print "Разбиение на пакеты \n"+str(self.frames)
    def __text2bin(cls,text):
        words = map(bin,bytearray(text))
        bitArStr = map(lambda x: x[2:],words)
        bitArStr = map(lambda x: '0'*(7-len(x))+x if len(x)<7 else x,bitArStr)
        bitArStr = "".join(bitArStr)
        bitArStr = BitArray("0b"+bitArStr)
        return bitArStr
    def __bin2text(cls, binaryStr):
        bin_to_word = binaryStr
        bin_to_word = list(map(''.join, zip(*[iter(bin_to_word )]*7)))
        bin_to_word = map(lambda x: '0b'+x,bin_to_word)
        bin_to_word = map(lambda x: chr(int(x,base=2)),bin_to_word)
        text_restored = "".join(bin_to_word)
        return text_restored
    def __makePackets(self):
        p_size = 288 # standart packet size
        stream = self.binStream.bin
        p_num = int(math.ceil(len(stream)/float(p_size))) # number of frames
        frameCounterSize = self.frameCounterSize
 
        packs = list()
        for i in xrange(p_num):
            packs.append( stream[i*p_size:min(i*p_size+p_size,len(stream)+1)])
        framesCounters = list()
        for i in xrange(1,p_num+1):
            framesCounters.append(i+500+3*i)            
        framesCounters = map(lambda x: bin(x),framesCounters)
        framesCounters = map(lambda x: '0b'+'0'*(frameCounterSize-len(x[2:]))+x[2:] if len(x[2:])<frameCounterSize else x,framesCounters)
        framesCounters= map(lambda x: BitArray(x),framesCounters)
        self.frames = zip(framesCounters, packs)
    def __nullRegisters(self):
        self.reg1= BitArray('0b'+'0'*19) # 19 bit register
        self.reg2= BitArray('0b'+'0'*22) # 22 bit register
        self.reg3= BitArray('0b'+'0'*23) # 23 bit register
    
    def __clockLFSR1(self, inputBit =0):
        reg1XOR = int(((self.reg1>>5)^(self.reg1>>2)^(self.reg1>>1)^(self.reg1))[-1])^inputBit
        self.reg1 = self.reg1>>1
        self.reg1[0]= reg1XOR
        #reg1 = ((((reg>>5)^(reg>>2)^(reg>>1)^(reg)) & one) <<18) | (reg>>one)
        #return int(reg1[-1])
        ##return int(reg1[-1])
    def __clockLFSR2(self, inputBit =0):
        reg2XOR = int(((self.reg2>>1)^(self.reg2))[-1])^inputBit
        self.reg2 = self.reg2>>1
        self.reg2[0]= reg2XOR
        #return int(reg2[-1])
    def __clockLFSR3(self, inputBit =0):
        reg3XOR = int(((self.reg3>>15)^(self.reg3>>2)^(self.reg3>>1)^(self.reg3))[-1])^inputBit
        self.reg3 = self.reg3>>1
        self.reg3[0]= reg3XOR
        #return int(reg3[-1])
    def __majorityFunc(self):
        x = self.reg1[8]
        y = self.reg2[10]
        z = self.reg3[10]
        return (x&y)|(x&z)|(y&z)
    def __sessionInit(self):
        for i in range(0,len(self.key)):
            self.__clockLFSR1(self.key[i])
            self.__clockLFSR2(self.key[i])
            self.__clockLFSR3(self.key[i])
    def __frameInit(self, frameCounter):
        for i in range(0,self.frameCounterSize):
            self.__clockLFSR1(frameCounter[i])
            self.__clockLFSR2(frameCounter[i])
            self.__clockLFSR3(frameCounter[i])
    def __emptyClocks(self, numIt = 100): #initial clocks without output
        for i in range(0,numIt):
            major = self.__majorityFunc()
            if self.reg1[8]==major:
                self.__clockLFSR1()
            if self.reg2[10]==major:
                self.__clockLFSR2()
            if self.reg3[10]==major:
                self.__clockLFSR3()
    def __mClocks(self, numIt =None): # clocks for random stream
        if numIt is None:
            numIt = self.frameCounterSize
        output = BitArray()
        for i in range(0,numIt):
            rez = self.reg1[-1]^self.reg2[-1]^self.reg3[-1]
            output.append('0b'+str(int(rez)))
            major = self.__majorityFunc()
            if self.reg1[8]==major:
                self.__clockLFSR1()
            if self.reg2[10]==major:
                self.__clockLFSR2()
            if self.reg3[10]==major:
                self.__clockLFSR3()
        return output
    def A5encoding(self):
        self.__makePackets()
        self.__sessionInit()
        for j in range(0,len(self.frames)):
            frame = self.frames[j]
            self.__nullRegisters()
            self.__frameInit(frameCounter=frame[0])
            self.__emptyClocks()
            randomFrame = self.__mClocks(len(frame[1]))
            if self.verbose: print "Packet iter tests"
            if self.verbose: print "random generate sequence"
            if self.verbose: print randomFrame.bin
            if self.verbose:  print "old sequence"
            if self.verbose: print frame[1]
            frame = ( frame[0],  randomFrame^BitArray('0b'+frame[1]) )
            self.frames[j] = frame
            if self.verbose: print "New sequence"
            if self.verbose: print self.frames[j][1].bin
            if self.verbose: print self.frames
        self.__mergePackets()
        self._getEncodedText()
        
        print "================================================================"
        print "                                                                "
        print "================================================================"
        print "Initial text"
        print self.__bin2text(self.binStream.bin)
        print "================================================================"
        print "Initial text in binary"
        print self.binStream.bin
        print "New encoded text in binary"
        print self.encodedBinStream.bin 
        print "================================================================"
        print "Encoded text"
        #print self.__bin2text(c.encodedBinStream.bin)
        print self.textEncrypted
        
    def __mergePackets(self):
        for frame in self.frames:
            self.encodedBinStream.append(frame[1])
    def _getEncodedText(self):
        self.textEncrypted = self.__bin2text(self.encodedBinStream.bin) 


In [21]:
st = open('Kittens.txt', 'r').read()

In [22]:
c =A5cipher(verbose=False)
c.loadText(st)
c.A5encoding()

                                                                
Initial text
Three little kittens they lost their mittens,
And they began to cry,
Oh, mother dear, we sadly fear
We have lost our mittens
What! lost your mittens, you naughty kittens!
Then you shall have no pie.
Mee-ow, mee-ow, mee-ow.
No, you shall have no pie.

The three little kittens they found their mittens,
And they began to cry,
Oh, mother dear, see here, see here,
We have found our mittens
Put on your mittens, you silly kittens,
And you shall have some pie.
Mee-ow, mee-ow, mee-ow.
Oh, let us have some pie.

The three little kittens put on their mittens,
And soon ate up the pie;
Oh, mother dear, we greatly fear
We have soiled our mittens
What! soiled your mittens, you naughty kittens!
Then they began to sigh,
Mee-ow, mee-ow, mee-ow.
Then they began to sigh.

Initial text in binary
1010100110100011100101100101110010101000001101100110100111101001110100110110011001010100000110101111010011110100111010011001011101110111

In [23]:
d =A5cipher(verbose=False)
d.loadText(c.textEncrypted)
d.A5encoding()
d = c.textEncrypted

                                                                
Initial text
I]vy)s=V?>fk2dhs*PV`]\Z{e;\f+AP0_ IzXeP0]K$Y,C>H/bww\"k#MNDN
@P=jEFx{%/8&:XoE<^%4eyh,SLg'7As=CvS0b-^$!,504?J
7STYGp.Lo<jI  8K~R(uszfi'Hpi[^U5
vTmjs8c_w{T7)?IY&~5E'*gTo'2O8Y`n	v_i?<#,({}D6Io\RAxsGp+n(>9s}2O!K|@VE)\_*a	|W@<dY3)5We+jhxyb&	?"&8%WBVEt 8bfp .="og$s1P2-zDl	(Ti@C@V!XEk}3BobE^
`VqH.'#WU,f!@Y'=Dm 7sJG%5\F>LJVHv;yXz^j>$}Y%!]1AAy3~%m
CsA[W7Ya&["IxyVZ7]VWOau{jG->_s84?s~[rc(Z^LBGD{I<H597G[|g,G_>a>ve;T(\(+zvG,c<e1kHCjlgI'XfgTCc,*J9v48uh"1ElJgdhNBG>*X!xpZn|R -	!ASJ`as352w7_qk)0Qf
H"9,\v-4Dim%R,
w>J.{q7
Initial text in binary
100100100001011011101111011011110010101001111001101111010010101111111100100011010110011111101111101100110110101101100101100100110100011100110010111010101010100000000011101011011000001011101101110010110101111011000001111001010011101011101110111